#### Importa os pacotes

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

#### Siglas dos Times

In [2]:
de_para_siglas = pd.read_csv('de_para_siglas.csv').set_index("sigla").to_dict()['nome']

#### Separa as informações no padrão do nome do arquivo 

In [219]:
def parse_filename(file_name):
    fl_name = file_name.replace(".csv", "").split("_")
    
    away_team = str(fl_name[0])
    home_team = str(fl_name[2])
    
    date_game = datetime.strptime(fl_name[3] + fl_name[4] + fl_name[5], '%b%d%Y')
    
    stat_team = fl_name[6]
    
    for key in de_para_siglas.keys():
        away_team_full = away_team.replace(key, de_para_siglas[key])
        stat_team_full = stat_team.replace(key, de_para_siglas[key])
        home_team_full = home_team.replace(key, de_para_siglas[key])
    
    df = pd.DataFrame(columns=["home_team", "away_team",
                               "date", "team"])
    df.loc[0] = [home_team, away_team,
                 date_game, stat_team]
    
    df["home_full"] = df["home_team"].replace(de_para_siglas, regex=True)
    df["away_full"] = df["away_team"].replace(de_para_siglas, regex=True)
    df["team_full"] = df["team"].replace(de_para_siglas, regex=True)
    
    return(df)

#### Lê o dataframe e retorna sem a coluna extra

In [225]:
def load_df(file_name):
    df = pd.read_csv("dfs/" + file_name)
    df = df.drop(["Unnamed: 0"], axis=1)
    return(df)

#### Junta as estatísticas básicas com as avançadas

In [221]:
def join_basic_adv_df(file_name_prefix):
    df_adv = load_df(file_name_prefix + "_Advanced.csv")
    df_basic = load_df(file_name_prefix + "_Basic.csv")
    
    df_full = pd.concat([df_basic, df_adv.drop(["Starters", "MP"], axis=1)], axis=1)
    columns_to_numeric = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg']

    df_full["MP"] = df_full["MP"].str.replace(":", ".") 
    df_full[columns_to_numeric] = df_full[columns_to_numeric].apply(pd.to_numeric, errors='coerce')
    
    info = parse_filename(file_name)
    df_full[list(info.columns)] = pd.concat([info]*len(df_full), ignore_index=True)
    
    df_full = df_full[["Starters"]  + list(info.columns) + columns_to_numeric]
    
    return(df_full)

#### Teste para um jogo

In [226]:
file_name = "ATL_@_BKN_Apr_02_2017_ATL"
df_full = join_basic_adv_df(file_name)

In [227]:
df_full.head(5)

,Starters,home_team,away_team,date,team,home_full,away_full,team_full,MP,FG,...,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg
0,Dennis Schroder,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,40.52,7.0,...,2.3,10.7,6.2,43.1,2.4,2.4,20.0,26.9,76.0,92.0
1,Taurean Waller-Prince,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,36.08,5.0,...,2.6,12.1,7.0,11.4,2.7,2.7,0.0,17.3,98.0,90.0
2,Tim Hardaway,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,35.51,5.0,...,0.0,18.3,8.5,5.7,0.0,0.0,29.8,20.6,79.0,97.0
3,Dwight Howard,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,25.25,4.0,...,3.7,42.9,21.9,0.0,0.0,3.9,39.2,22.1,80.0,88.0
4,Ersan Ilyasova,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,20.18,1.0,...,0.0,0.0,0.0,17.1,2.4,4.8,14.5,14.9,70.0,93.0


#### Carrega as lista de arquivos de todos os jogos

In [228]:
directory = "dfs/"

files_list = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        file_name = filename.replace("_Basic.csv", "")
        file_name = file_name.replace("_Advanced.csv", "")
        files_list.append(file_name)
        
files_list = list(dict.fromkeys(files_list))

#### Gera DataFrame completo com informações

In [231]:
df_resp = None

for file_name in files_list:
    if(df_resp is None):
        df_resp = join_basic_adv_df(file_name)
    else:
        df_resp = pd.concat([df_resp, join_basic_adv_df(file_name)], axis=0).reset_index(drop=True)

In [243]:
df_resp['year_month'] = df_resp['date'].map(lambda x: 100*x.year + x.month)
df_resp["game"] = df_resp['away_team'] + " @ " + df_resp['home_team'] + " " + df_resp["date"].map(lambda x: x.strftime('%Y-%m-%d'))

In [245]:
df_resp.to_csv("nba_games_dataset.csv")

In [246]:
df_resp.columns

Index(['Starters', 'home_team', 'away_team', 'date', 'team', 'home_full',
       'away_full', 'team_full', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'year_month', 'game'],
      dtype='object')

In [248]:
df_resp.head()

,Starters,home_team,away_team,date,team,home_full,away_full,team_full,MP,FG,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,year_month,game
0,Dennis Schroder,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,40.52,7.0,...,6.2,43.1,2.4,2.4,20.0,26.9,76.0,92.0,201704,ATL @ BKN 2017-04-02
1,Taurean Waller-Prince,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,36.08,5.0,...,7.0,11.4,2.7,2.7,0.0,17.3,98.0,90.0,201704,ATL @ BKN 2017-04-02
2,Tim Hardaway,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,35.51,5.0,...,8.5,5.7,0.0,0.0,29.8,20.6,79.0,97.0,201704,ATL @ BKN 2017-04-02
3,Dwight Howard,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,25.25,4.0,...,21.9,0.0,0.0,3.9,39.2,22.1,80.0,88.0,201704,ATL @ BKN 2017-04-02
4,Ersan Ilyasova,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,20.18,1.0,...,0.0,17.1,2.4,4.8,14.5,14.9,70.0,93.0,201704,ATL @ BKN 2017-04-02


In [235]:
# Basic Box Score Stats

# MP -- Minutes Played
# FG -- Field Goals
# FGA -- Field Goal Attempts
# FG% -- Field Goal Percentage
# 3P -- 3-Point Field Goals
# 3PA -- 3-Point Field Goal Attempts
# 3P% -- 3-Point Field Goal Percentage
# FT -- Free Throws
# FTA -- Free Throw Attempts
# FT% -- Free Throw Percentage
# ORB -- Offensive Rebounds
# DRB -- Defensive Rebounds
# TRB -- Total Rebounds
# AST -- Assists
# STL -- Steals
# BLK -- Blocks
# TOV -- Turnovers
# PF -- Personal Fouls
# PTS -- Points
# +/- -- Plus/Minus

# Advanced Box Score Stats
# MP -- Minutes Played
# TS% -- True Shooting Percentage
# A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.
# eFG% -- Effective Field Goal Percentage
# This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.
# 3PAr -- 3-Point Attempt Rate
# Percentage of FG Attempts from 3-Point Range
# FTr -- Free Throw Attempt Rate
# Number of FT Attempts Per FG Attempt
# ORB% -- Offensive Rebound Percentage
# An estimate of the percentage of available offensive rebounds a player grabbed while he was on the floor.
# DRB% -- Defensive Rebound Percentage
# An estimate of the percentage of available defensive rebounds a player grabbed while he was on the floor.
# TRB% -- Total Rebound Percentage
# An estimate of the percentage of available rebounds a player grabbed while he was on the floor.
# AST% -- Assist Percentage
# An estimate of the percentage of teammate field goals a player assisted while he was on the floor.
# STL% -- Steal Percentage
# An estimate of the percentage of opponent possessions that end with a steal by the player while he was on the floor.
# BLK% -- Block Percentage
# An estimate of the percentage of opponent two-point field goal attempts blocked by the player while he was on the floor.
# TOV% -- Turnover Percentage
# An estimate of turnovers committed per 100 plays.
# USG% -- Usage Percentage
# An estimate of the percentage of team plays used by a player while he was on the floor.
# ORtg -- Offensive Rating
# An estimate of points produced (players) or scored (teams) per 100 possessions
# DRtg -- Defensive Rating
# An estimate of points allowed per 100 possessions

In [236]:
columns_sum = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', '3PAr', 'FTr', 'ORtg', 'DRtg']
columns_avg = ['FG%', '3P%', 'FT%', 'TS%', 'eFG%', 'ORB%', 'DRB%',
              'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
              ]

In [249]:
game_totals = df_resp[df_resp.Starters == "Team Totals"].groupby(["game", "team"], as_index=False).sum()

In [251]:
game_totals.head()

,game,team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,year_month
0,ATL @ BKN 2015-11-17,ATL,240.0,33.0,76.0,0.434,12.0,29.0,0.414,10.0,...,80.0,50.0,66.7,7.6,10.9,19.6,100.0,96.1,98.3,201511
1,ATL @ BKN 2015-11-17,BKN,240.0,37.0,84.0,0.440,9.0,20.0,0.450,7.0,...,81.4,50.0,59.5,15.3,6.4,10.9,100.0,98.3,96.1,201511
2,ATL @ BKN 2017-01-10,ATL,240.0,44.0,92.0,0.478,7.0,23.0,0.304,22.0,...,73.1,52.5,56.8,10.9,20.7,9.9,100.0,116.0,96.2,201701
3,ATL @ BKN 2017-01-10,BKN,240.0,35.0,87.0,0.402,10.0,29.0,0.345,17.0,...,69.4,47.5,62.9,5.0,5.8,14.9,100.0,96.2,116.0,201701
4,ATL @ BKN 2017-04-02,ATL,240.0,30.0,80.0,0.375,7.0,24.0,0.292,15.0,...,81.8,46.3,63.3,9.0,8.2,17.4,100.0,82.4,91.4,201704


In [252]:
game_totals.to_csv("game_totals.csv")

In [4]:
game_totals = pd.read_csv("game_totals.csv")
game_totals = game_totals.drop(["Unnamed: 0"], axis=1)

In [253]:
game_totals["fl_home"] = np.where(game_totals["game"].str[6:9] == game_totals["team"], 1, 0)

In [254]:
game_totals.columns

Index(['game', 'team', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'year_month',
       'fl_home'],
      dtype='object')

In [259]:
home_games = game_totals[game_totals["fl_home"] == 1].set_index("game")
away_games = game_totals[game_totals["fl_home"] == 0].set_index("game")

In [261]:
all_games.columns

Index(['team_home', 'MP_home', 'FG_home', 'FGA_home', 'FG%_home', '3P_home',
       '3PA_home', '3P%_home', 'FT_home', 'FTA_home', 'FT%_home', 'ORB_home',
       'DRB_home', 'TRB_home', 'AST_home', 'STL_home', 'BLK_home', 'TOV_home',
       'PF_home', 'PTS_home', '+/-_home', 'TS%_home', 'eFG%_home', '3PAr_home',
       'FTr_home', 'ORB%_home', 'DRB%_home', 'TRB%_home', 'AST%_home',
       'STL%_home', 'BLK%_home', 'TOV%_home', 'USG%_home', 'ORtg_home',
       'DRtg_home', 'year_month_home', 'fl_home_home', 'team_away', 'MP_away',
       'FG_away', 'FGA_away', 'FG%_away', '3P_away', '3PA_away', '3P%_away',
       'FT_away', 'FTA_away', 'FT%_away', 'ORB_away', 'DRB_away', 'TRB_away',
       'AST_away', 'STL_away', 'BLK_away', 'TOV_away', 'PF_away', 'PTS_away',
       '+/-_away', 'TS%_away', 'eFG%_away', '3PAr_away', 'FTr_away',
       'ORB%_away', 'DRB%_away', 'TRB%_away', 'AST%_away', 'STL%_away',
       'BLK%_away', 'TOV%_away', 'USG%_away', 'ORtg_away', 'DRtg_away',
       'year_month

In [263]:
all_games = home_games.join(away_games, how="inner", lsuffix="_home", rsuffix="_away")
all_games.drop(["year_month_home", "year_month_away", "fl_home_away",
                "fl_home_home", "MP_away", "fl_home_away", "fl_home_home", "+/-", "+/-_home"], axis=1, inplace=True)

In [264]:
all_games.columns

Index(['team_home', 'MP_home', 'FG_home', 'FGA_home', 'FG%_home', '3P_home',
       '3PA_home', '3P%_home', 'FT_home', 'FTA_home', 'FT%_home', 'ORB_home',
       'DRB_home', 'TRB_home', 'AST_home', 'STL_home', 'BLK_home', 'TOV_home',
       'PF_home', 'PTS_home', '+/-_home', 'TS%_home', 'eFG%_home', '3PAr_home',
       'FTr_home', 'ORB%_home', 'DRB%_home', 'TRB%_home', 'AST%_home',
       'STL%_home', 'BLK%_home', 'TOV%_home', 'USG%_home', 'ORtg_home',
       'DRtg_home', 'team_away', 'FG_away', 'FGA_away', 'FG%_away', '3P_away',
       '3PA_away', '3P%_away', 'FT_away', 'FTA_away', 'FT%_away', 'ORB_away',
       'DRB_away', 'TRB_away', 'AST_away', 'STL_away', 'BLK_away', 'TOV_away',
       'PF_away', 'PTS_away', '+/-_away', 'TS%_away', 'eFG%_away', '3PAr_away',
       'FTr_away', 'ORB%_away', 'DRB%_away', 'TRB%_away', 'AST%_away',
       'STL%_away', 'BLK%_away', 'TOV%_away', 'USG%_away', 'ORtg_away',
       'DRtg_away'],
      dtype='object')

In [265]:
all_games.head()

,team_home,MP_home,FG_home,FGA_home,FG%_home,3P_home,3PA_home,3P%_home,FT_home,FTA_home,...,ORB%_away,DRB%_away,TRB%_away,AST%_away,STL%_away,BLK%_away,TOV%_away,USG%_away,ORtg_away,DRtg_away
game,,,,,,,,,,,,,,,,,,,,,
ATL @ BKN 2015-11-17,BKN,240.0,37.0,84.0,0.440,9.0,20.0,0.450,7.0,13.0,...,18.6,80.0,50.0,66.7,7.6,10.9,19.6,100.0,96.1,98.3
ATL @ BKN 2017-01-10,BKN,240.0,35.0,87.0,0.402,10.0,29.0,0.345,17.0,23.0,...,30.6,73.1,52.5,56.8,10.9,20.7,9.9,100.0,116.0,96.2
ATL @ BKN 2017-04-02,BKN,240.0,32.0,82.0,0.390,10.0,33.0,0.303,17.0,20.0,...,15.7,81.8,46.3,63.3,9.0,8.2,17.4,100.0,82.4,91.4
ATL @ BKN 2017-10-22,BKN,240.0,41.0,87.0,0.471,11.0,27.0,0.407,23.0,34.0,...,29.6,71.1,48.5,65.6,5.7,3.3,12.7,100.0,99.5,110.9
ATL @ BKN 2017-12-02,BKN,240.0,34.0,82.0,0.415,17.0,42.0,0.405,17.0,21.0,...,27.7,73.9,50.5,38.3,12.4,12.5,7.1,100.0,118.0,105.6


In [266]:
def get_season(date):
    ano = date.year
    if(date.month >= 10):
        return ano + 1
    return ano

In [281]:
def is_playoff(date):
    # Playoffs 2016
    if date >= datetime(2016, 4, 16) and date < datetime(2016, 6, 30):
        return 1
    
    # Playoffs 2017
    elif date >= datetime(2017, 4, 15) and date < datetime(2017, 6, 30):
        return 1
    
     # Playoffs 2018
    elif date >= datetime(2018, 4, 14) and date < datetime(2018, 6, 30):
        return 1
    
    return 0

In [292]:
all_games["date"] = [datetime.strptime(str(x)[10:20], '%Y-%m-%d') for x in all_games.index]
all_games["season"] = [get_season(x) for x in all_games.date]
all_games["fl_playoff"] = [is_playoff(x) for x in all_games.date]
all_games = all_games.sort_values('date')

In [294]:
all_games["team_home_game_num"] = all_games.groupby(['team_home', 'season']).cumcount() + 1
all_games["team_away_game_num"] = all_games.groupby(['team_away', 'season']).cumcount() + 1

In [295]:
all_games.head()

,team_home,MP_home,FG_home,FGA_home,FG%_home,3P_home,3PA_home,3P%_home,FT_home,FTA_home,...,BLK%_away,TOV%_away,USG%_away,ORtg_away,DRtg_away,date,season,team_home_game_num,team_away_game_num,fl_playoff
game,,,,,,,,,,,,,,,,,,,,,
NOP @ GSW 2015-10-27,GSW,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,...,4.5,15.9,100.0,94.9,110.9,2015-10-27,2016,1,1,0
CLE @ CHI 2015-10-27,CHI,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,10.3,9.0,100.0,95.5,97.5,2015-10-27,2016,1,1,0
DET @ ATL 2015-10-27,ATL,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,5.5,12.3,100.0,111.2,98.6,2015-10-27,2016,1,1,0
CLE @ MEM 2015-10-28,MEM,240.0,29.0,82.0,0.354,2.0,16.0,0.125,16.0,22.0,...,3.0,17.2,100.0,110.8,79.4,2015-10-28,2016,1,2,0
CHA @ MIA 2015-10-28,MIA,240.0,36.0,73.0,0.493,12.0,20.0,0.600,20.0,21.0,...,1.9,7.6,100.0,102.4,113.2,2015-10-28,2016,1,1,0


In [319]:
all_games[["team_home", "team_away", "date", "team_home_game_num", "team_away_game_num"]].head()

,team_home,team_away,date,team_home_game_num,team_away_game_num
game,,,,,
NOP @ GSW 2015-10-27,GSW,NOP,2015-10-27,1,1
CLE @ CHI 2015-10-27,CHI,CLE,2015-10-27,1,1
DET @ ATL 2015-10-27,ATL,DET,2015-10-27,1,1
CLE @ MEM 2015-10-28,MEM,CLE,2015-10-28,1,2
CHA @ MIA 2015-10-28,MIA,CHA,2015-10-28,1,1


In [84]:
teste_rolling = all_games[all_games["team_home"] == "ATL"].groupby('season')['PTS_home'].apply(lambda x:x.rolling(center=False,window=5).mean())

In [307]:
all_games[all_games["fl_playoff"] == 0].groupby('season').describe()

+/-_away                              +/-_home       ...   \
          count mean std min 25% 50% 75% max    count mean  ...    
season                                                      ...    
2016        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    
2017        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    
2018        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    

       team_away_game_num       team_home_game_num                        \
                      75%   max              count  mean        std  min   
season                                                                     
2016                 31.0  41.0             1230.0  21.0  11.836972  1.0   
2017                 31.0  41.0             1230.0  21.0  11.836972  1.0   
2018                 31.0  41.0             1230.0  21.0  11.836972  1.0   

                                
         25%   50%   75%   max  
season                          
2016    11.0  21.0  31.0  41.0  
2017    11.0  21.0  31.0  41.0  
2018    11.0  21.0  31.0  41.0  

[3 rows x 560 columns]

### Método que retorna a média dos últimos N jogos

In [6]:
def get_last_games(df, data, team_name, n = 5, filter="all", verbose=False):
    if(filter == "all"):
        last_games = df[(df["date"] < data) & 
                        ((df["team_home"] == team_name) | (df["team_away"] == team_name))].tail(n)
        
    elif(filter == "home"):
        last_games = df[(df["date"] < data) & (df["team_home"] == team_name)].tail(n)
    elif(filter == "away"):
        last_games = df[(df["date"] < data) & (df["team_away"] == team_name)].tail(n)
    
    home_columns = [x for x in all_games.columns if x.endswith("_home")]
    away_columns = [x for x in all_games.columns if x.endswith("_away")]
    
    if(verbose):
        print(last_games[["team_home", "team_away", "PTS_home", "PTS_away"]])
    
    last_games_home = last_games[last_games["team_home"] == team_name][home_columns].groupby(["team_home"]).sum()
    last_games_home.columns = [x.replace("_home","") for x in last_games_home.columns]
    
    last_games_away = last_games[last_games["team_away"] == team_name][away_columns].groupby(["team_away"]).sum()
    last_games_away.columns = [x.replace("_away","") for x in last_games_away.columns]
    
    if(len(last_games_home) == 0):
        return(last_games_away/n)
    
    if(len(last_games_away) == 0):
        return(last_games_home/n)
    
    return((last_games_home + last_games_away) / n)

In [297]:
teste = all_games[all_games["team_home"] == "ATL"]

In [298]:
all_games[(all_games["date"] < teste.iloc[10]["date"]) 
          & ((all_games["team_home"] == "ATL") | (all_games["team_away"] == "ATL"))].tail(5)[["team_home", "team_away", "PTS_home", "PTS_away"]]

,team_home,team_away,PTS_home,PTS_away
game,,,,
BOS @ ATL 2015-11-24,ATL,BOS,121.0,97.0
ATL @ MIN 2015-11-25,MIN,ATL,99.0,95.0
ATL @ MEM 2015-11-27,MEM,ATL,101.0,116.0
ATL @ SAS 2015-11-28,SAS,ATL,108.0,88.0
OKC @ ATL 2015-11-30,ATL,OKC,106.0,100.0


In [343]:
get_last_games(all_games, teste.iloc[10]["date"], "ATL", verbose=True).reset_index()

                     team_home team_away  PTS_home  PTS_away
game                                                        
BOS @ ATL 2015-11-24       ATL       BOS     121.0      97.0
ATL @ MIN 2015-11-25       MIN       ATL      99.0      95.0
ATL @ MEM 2015-11-27       MEM       ATL     101.0     116.0
ATL @ SAS 2015-11-28       SAS       ATL     108.0      88.0
OKC @ ATL 2015-11-30       ATL       OKC     106.0     100.0


,team_home,+/-,3P,3P%,3PA,3PAr,AST,AST%,BLK,BLK%,...,PTS,STL,STL%,TOV,TOV%,TRB,TRB%,TS%,USG%,eFG%
0,ATL,NaN,10.0,0.341,29.6,0.3616,24.6,64.92,6.0,9.44,...,105.2,8.0,8.32,14.8,13.56,46.0,52.16,0.5624,100.0,0.5148


In [302]:
get_last_games(all_games, teste.iloc[10]["date"], "ATL", verbose=True, filter="away")

                     team_home team_away  PTS_home  PTS_away
game                                                        
ATL @ BKN 2015-11-17       BKN       ATL      90.0      88.0
ATL @ CLE 2015-11-21       CLE       ATL     109.0      97.0
ATL @ MIN 2015-11-25       MIN       ATL      99.0      95.0
ATL @ MEM 2015-11-27       MEM       ATL     101.0     116.0
ATL @ SAS 2015-11-28       SAS       ATL     108.0      88.0


,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,...,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg
team_away,,,,,,,,,,,,,,,,,,,,,
ATL,33.6,81.2,0.4152,10.2,30.6,0.331,19.4,24.6,0.7624,10.6,...,22.06,78.8,49.08,65.56,8.48,8.58,13.88,100.0,102.8,107.58


In [309]:
all_games.to_csv("all_games.csv")

### Gera a Base com as médias móveis por jogo

In [3]:
all_games = pd.read_csv("all_games.csv")

In [11]:
all_games.set_index("game", inplace=True)

In [4]:
regular_season = all_games[all_games["fl_playoff"] == 0]
len(regular_season)/3

1230.0

In [15]:
resp = []

for season_df in regular_season.groupby("season"):
    for index, row in season_df[1].iterrows():
        home_last_games = get_last_games(season_df[1], row["date"], row["team_home"]).reset_index()
        home_last_games["game_ref"] = [row.game]
        home_last_games.set_index("game_ref")
        home_last_games.drop(["team_home", "team_away"],axis=1 ,errors="ignore", inplace=True)
        
        away_last_games = get_last_games(season_df[1], row["date"], row["team_away"]).reset_index()
        away_last_games["game_ref"] = [row.game]
        away_last_games.set_index("game_ref")
        away_last_games.drop(["team_home", "team_away"],axis=1 ,errors="ignore", inplace=True)
        
        game_line = home_last_games.join(away_last_games, how="inner", 
                             lsuffix='_home_last_5_games', rsuffix='_away_last_5_games')
        game_line = game_line.set_index('game_ref_home_last_5_games')
        
        print(row.game)
        
        resp.append(game_line)

NOP @ GSW 2015-10-27
CLE @ CHI 2015-10-27
DET @ ATL 2015-10-27
CLE @ MEM 2015-10-28
CHA @ MIA 2015-10-28
NOP @ POR 2015-10-28
DAL @ PHX 2015-10-28
PHI @ BOS 2015-10-28
CHI @ BKN 2015-10-28
SAS @ OKC 2015-10-28
WAS @ ORL 2015-10-28
UTA @ DET 2015-10-28
DEN @ HOU 2015-10-28
IND @ TOR 2015-10-28
LAC @ SAC 2015-10-28
MIN @ LAL 2015-10-28
NYK @ MIL 2015-10-28
MEM @ IND 2015-10-29
ATL @ NYK 2015-10-29
DAL @ LAC 2015-10-29
UTA @ PHI 2015-10-30
POR @ PHX 2015-10-30
MIN @ DEN 2015-10-30
LAL @ SAC 2015-10-30
CHA @ ATL 2015-10-30
BKN @ SAS 2015-10-30
CHI @ DET 2015-10-30
GSW @ HOU 2015-10-30
MIA @ CLE 2015-10-30
TOR @ BOS 2015-10-30
OKC @ ORL 2015-10-30
WAS @ MIL 2015-10-30
NYK @ WAS 2015-10-31
PHX @ POR 2015-10-31
GSW @ NOP 2015-10-31
BKN @ MEM 2015-10-31
UTA @ IND 2015-10-31
SAC @ LAC 2015-10-31
ORL @ CHI 2015-11-01
DAL @ LAL 2015-11-01
DEN @ OKC 2015-11-01
HOU @ MIA 2015-11-01
MIL @ TOR 2015-11-01
SAS @ BOS 2015-11-01
ATL @ CHA 2015-11-01
CLE @ PHI 2015-11-02
POR @ MIN 2015-11-02
MEM @ GSW 201

In [18]:
last_all_games = pd.concat(resp)

In [27]:
last_all_games.drop(["game_ref_away_last_5_games", "+/-_away_last_5_games", "+/-_home_last_5_games"], axis = 1, inplace=True)

In [31]:
last_all_games.head()

,3P%_away_last_5_games,3P%_home_last_5_games,3PA_away_last_5_games,3PA_home_last_5_games,3PAr_away_last_5_games,3PAr_home_last_5_games,3P_away_last_5_games,3P_home_last_5_games,AST%_away_last_5_games,AST%_home_last_5_games,...,TRB%_away_last_5_games,TRB%_home_last_5_games,TRB_away_last_5_games,TRB_home_last_5_games,TS%_away_last_5_games,TS%_home_last_5_games,USG%_away_last_5_games,USG%_home_last_5_games,eFG%_away_last_5_games,eFG%_home_last_5_games
game_ref_home_last_5_games,,,,,,,,,,,,,,,,,,,,,
NOP @ GSW 2015-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CLE @ CHI 2015-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DET @ ATL 2015-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CLE @ MEM 2015-10-28,0.062,NaN,5.8,NaN,0.0618,NaN,1.8,NaN,13.68,NaN,...,10.3,NaN,10.0,NaN,0.0936,NaN,20.0,NaN,0.0904,NaN
CHA @ MIA 2015-10-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
base_full = all_games.join(last_all_games, how="inner").drop(["+/-_home", "+/-_away", "MP", "MP_home",
                                                              "MP_away_last_5_games", "MP_home_last_5_games"], axis=1)

In [44]:
base_full['fl_home_win'] = np.where(base_full['PTS_home'] > base_full['PTS_away'], 1, 0)

In [46]:
print(list(base_full.columns))

['team_home', 'FG_home', 'FGA_home', 'FG%_home', '3P_home', '3PA_home', '3P%_home', 'FT_home', 'FTA_home', 'FT%_home', 'ORB_home', 'DRB_home', 'TRB_home', 'AST_home', 'STL_home', 'BLK_home', 'TOV_home', 'PF_home', 'PTS_home', 'TS%_home', 'eFG%_home', '3PAr_home', 'FTr_home', 'ORB%_home', 'DRB%_home', 'TRB%_home', 'AST%_home', 'STL%_home', 'BLK%_home', 'TOV%_home', 'USG%_home', 'ORtg_home', 'DRtg_home', 'team_away', 'FG_away', 'FGA_away', 'FG%_away', '3P_away', '3PA_away', '3P%_away', 'FT_away', 'FTA_away', 'FT%_away', 'ORB_away', 'DRB_away', 'TRB_away', 'AST_away', 'STL_away', 'BLK_away', 'TOV_away', 'PF_away', 'PTS_away', 'TS%_away', 'eFG%_away', '3PAr_away', 'FTr_away', 'ORB%_away', 'DRB%_away', 'TRB%_away', 'AST%_away', 'STL%_away', 'BLK%_away', 'TOV%_away', 'USG%_away', 'ORtg_away', 'DRtg_away', 'date', 'season', 'team_home_game_num', 'team_away_game_num', 'fl_playoff', '3P%_away_last_5_games', '3P%_home_last_5_games', '3PA_away_last_5_games', '3PA_home_last_5_games', '3PAr_away_la

In [48]:
base_full.to_csv("base_full.csv")